Week5 Advanced 심화과제

- 논문요약

    1) 필수 지시사항은 논문을 요약하는 것입니다.

    2) 이를 조금 확장시켜, rag에 관한 여러가지 논문을 읽어와 rag에 관한 질문은 검색된 문서 기반으로 대답하고 일상 대화는 context에 상관없이 대답하는 챗봇을 만들 예정입니다.

In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv(dotenv_path="config.env")
openai.api_key = os.getenv("OPENAI_API_KEY")

## 1 [My Code] Preparation

### 1.1 [My Code] Data Load

- pdf를 불러올 땐 unstructuredfileloader를 사용합니다

In [2]:
from pathlib import Path

from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# pdf를 불러올 땐 unstructuredFileLoader를 사용합니다
path = Path('./data/essay')
documents = [UnstructuredFileLoader(file_path).load() for file_path in path.glob('*.pdf')]


/tmp/ipykernel_1079/3843052403.py:8: LangChainDeprecationWarning: The class `UnstructuredFileLoader` was deprecated in LangChain 0.2.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-unstructured package and should be used instead. To use it run `pip install -U :class:`~langchain-unstructured` and import as `from :class:`~langchain_unstructured import UnstructuredLoader``.
  documents = [UnstructuredFileLoader(file_path).load() for file_path in path.glob('*.pdf')]


### 1.2 [My Code] TextSplitter

In [3]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splitted_docs = []
for document in documents:
    splitted_docs.extend(splitter.split_documents(document))

### 1.3 [My Code] 문장 생성 모델 선언

In [4]:
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain_openai import ChatOpenAI

# 문장 생성기
llm = ChatOpenAI(
    model='gpt-4o-mini',
    temperature=0.7,
    max_tokens=1024,
    )

### 1.4 [My Code] 임베딩 모델 선언

- 여기선 CacheBackedEmbeddings을 사용하여, 인풋이 같을 때엔 다시 임베딩을 하지 않고 캐시된 정보를 사용하도록 수정합니다.,

In [5]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain.embeddings import CacheBackedEmbeddings
from langchain.storage import LocalFileStore


# OpenAI Embeddings 초기화
# https://platform.openai.com/docs/guides/embeddings
# 임베딩 전용
embeddings = OpenAIEmbeddings(
    api_key=openai.api_key,
    model="text-embedding-3-small"
)

# 이렇게 하면 동일한 파일에 대해서, 매번 임베딩하지 않습니다.
cache_dir = LocalFileStore(".cache/files")
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

# Chroma 벡터 저장소 생성 및 로컬 저장 경로 지정
vectorstore = Chroma.from_documents(
    documents=splitted_docs,
    embedding=cached_embeddings,
    collection_name="my_db"
)


# 유사 문서는 20개를 찾도록 합니다.
chroma_retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k': 20})

### 1.5 [My Code] 프롬프트

- LLM 교수와 학생의 질의응답 상황을 가정합니다.

- rag와 관련 없는 질문은 context와 상관없이 대답하며 rag와 관련된 질문은 context 내에서 답변하도록 prompt를 구성합니다.

In [6]:
from langchain.prompts import ChatPromptTemplate

prompt_template = """
당신은 대학 LLM 교수입니다. 아래 지침에 따라 학생의 질문에 답해주세요.

1. 문맥에서 답을 명확히 찾아내야 하며, **rag(Relevant Answer Generation)**에 관한 질문은 반드시 제공된 문맥에만 근거해서 답변해야 합니다.  
   - rag에 관한 질문에서 문맥만으로 답을 찾을 수 없으면, "아직 저도 부족해서 공부가 더 필요할 거 같아요. 같이 공부해봐요."라고 대답하세요.  
2. rag와 관련 없는 일상적인 대화에는 자유롭게 답변하되, 친절하고 대화체로 작성하세요.  
3. 모든 답변은 5문장을 넘지 않아야 하며, rag 관련 답변에는 반드시 "잘 이해되었나요?"라는 질문을 포함하세요.  
4. 학생이 이해하기 쉽게 설명하며, 항상 친절한 교수님의 톤을 유지하세요.

문맥:  
{context}

학생의 질문:  
{question}

답변:
"""
chat_prompt_template = ChatPromptTemplate.from_template(prompt_template)

## 2 [My Code] 챗봇

- 문맥은 5개만 기억하도록 설정합니다.

- streamlit 등에선 따로 session이 존재하지만 파이썬 자체로 구현하기 위해 deque 데이터타입을 사용합니다.

In [12]:
from collections import deque

class ChatBot:
    def __init__(self, retriever, llm, prompt_template, history_limit=5):
        self.retriever = retriever
        self.llm = llm
        self.prompt_template = prompt_template
        self.history = deque(maxlen=history_limit)

    # docs: Document 객체의 list
    @staticmethod
    def _format_docs(docs):
        return "\n\n".join(f"Essay Source: {doc.metadata['source'].split('/')[-1]}, Content: {doc.page_content}" for doc in docs)
    
    def create_prompt(self, user_message):
        # 처음엔 context가 없어서 None으로 들어갑니다.
        # 따라서 초반 서사가 쌓이기 전까진 retrieved_docs만 들어갑니다.
        context = "\n\n".join([f"Student: {msg['Student']}\nLLM Professor: {msg['LLM Professor']}" for msg in self.history])
        retrieved_docs = self.retriever.invoke(user_message)

        # 반환된 document를 전처리합니다.
        retrieved_context = ChatBot._format_docs(retrieved_docs)
        
        return self.prompt_template.invoke({
            "context": context + "\n\n" + retrieved_context,
            "question": user_message
        })

    def chat(self, user_message):
        user_message = user_message.lower()
        user_prompt = self.create_prompt(user_message)
        response = self.llm.invoke(user_prompt)
        self.history.append({"Student": user_message, "LLM Professor": response})
        return response
    
chatbot = ChatBot(retriever=chroma_retriever, llm=llm, prompt_template=chat_prompt_template)
print("LLM 수업 시작", end="\n\n")
while True:
    user_input = input("Student: ")
    if user_input.lower() in ["종료!"]:
        print("LLM Professor: 오늘 수업은 여기서 마치겠습니다.", end="\n\n")
        print("LLM 수업 끝", end="\n\n")
        break
    response = chatbot.chat(user_input)
    print(f"Student: {user_input}", end="\n\n")
    print(f"LLM Professor: {response.content}", end="\n\n")

LLM 수업 시작

Student: 안녕하세요! 오늘 수업 과목은 무엇인가요?

LLM Professor: 안녕하세요! 오늘 수업은 Retrieval-Augmented Generation(RAG)에 대한 내용을 다룰 예정이에요. RAG의 기본 개념과 성능 평가, 그리고 다양한 응용 분야에 대해 이야기할 거예요. 이 기술이 어떻게 LLM의 성능을 향상시킬 수 있는지, 그리고 최신 연구 동향도 살펴볼 거예요. 궁금한 점이 있으면 언제든지 물어보세요!

Student: rag의 정의는 뭔가요?

LLM Professor: RAG, 즉 Retrieval-Augmented Generation은 사용자의 입력 쿼리를 기반으로 외부 데이터베이스에서 정보를 검색하고, 이를 활용하여 응답을 생성하는 기술입니다. 이 과정은 크게 세 가지 요소로 구성됩니다: 인덱싱, 검색, 생성입니다. RAG는 LLM(대형 언어 모델)의 성능을 향상시키는 데 도움을 주며, 특히 최신 정보에 대한 접근성을 높입니다. 이를 통해 보다 정확하고 관련성 높은 응답을 제공할 수 있습니다. 잘 이해되었나요?

Student: 잘 이해 되었어요, 오늘 점심은 뭐먹을까요?

LLM Professor: 점심 메뉴를 고르는 건 항상 즐거운 고민이죠! 혹시 좋아하는 음식이 있으신가요? 아니면 새로운 음식을 시도해보고 싶으신가요? 요즘 인기 있는 메뉴로는 샌드위치나 파스타, 또는 건강한 샐러드도 좋을 것 같아요. 함께 고민해보면 좋을 것 같아요! 어떤 게 마음에 드세요?

Student: rag는 rapid align generation의 약자 인가요?

LLM Professor: RAG는 Rapid Align Generation의 약자가 아니라 Retrieval-Augmented Generation의 약자입니다. RAG는 사용자의 입력을 기반으로 외부 데이터베이스에서 정보를 검색하고, 이를 통해 응답을 생성하는 기술입니다. 이 과정에서 RAG는 LLM의 성능을 향상시키고 최신 정보에 대한 접근성을 높여줍니다. 혹시